In [1]:
import tensorflow as tf

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
os.environ["NCCL_DEBUG"] = "INFO"
# # export CUDA_HOME='/usr/local/cuda-10.1'
# os.environ['CUDA_HOME'] ='/usr/local/cuda-11.5'
# os.environ['PATH'] = '/usr/local/cuda-11.5/bin:' + os.environ['PATH']
# os.environ['LIBRARY_PATH'] = '/usr/local/cuda-11.5/lib64:' + os.environ['LIBRARY_PATH']
# os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.5/lib64:'+ os.environ['LD_LIBRARY_PATH']
# 使用设置好的环境变量
# 加载 CIFAR-10 数据集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# 归一化图像数据
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 将目标变量转换为 one-hot 编码
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 定义一个简单的 CNN 模型
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model

# 使用 MirroredStrategy 进行多 GPU 训练
strategy = tf.distribute.MirroredStrategy()

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# 在策略范围内创建和编译模型
with strategy.scope():
    model = create_model()  
    
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))


2024-01-06 07:16:44.586897: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-06 07:16:45.769118: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


2024-01-06 07:16:54.515070: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11421 MB memory:  -> device: 0, name: NVIDIA TITAN X (Pascal), pci bus id: 0000:04:00.0, compute capability: 6.1
2024-01-06 07:16:54.516319: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 11421 MB memory:  -> device: 1, name: NVIDIA TITAN X (Pascal), pci bus id: 0000:05:00.0, compute capability: 6.1
2024-01-06 07:16:54.517344: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 11421 MB memory:  -> device: 2, name: NVIDIA TITAN X (Pascal), pci bus id: 0000:08:00.0, compute capability: 6.1
2024-01-06 07:16:54.518313: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 11421 MB memory:  -> device: 3, name: NVIDIA TITAN X (Pascal)

Number of devices: 8
Epoch 1/10
INFO:tensorflow:Collective all_reduce tensors: 8 all_reduces, num_devices = 8, group_size = 8, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Collective all_reduce tensors: 8 all_reduces, num_devices = 8, group_size = 8, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to

2024-01-06 07:17:09.277208: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-01-06 07:17:10.374094: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-01-06 07:17:11.511285: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-01-06 07:17:12.440728: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-01-06 07:17:13.077992: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-01-06 07:17:13.781718: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


...

2024-01-06 07:17:14.703302: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-01-06 07:17:15.504906: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


soft:18442:18979 [0] NCCL INFO cudaDriverVersion 11060
NCCL version 2.13.4+cudaCUDA_MAJOR.CUDA_MINOR
soft:18442:18994 [0] NCCL INFO NET/IB : No device found.
soft:18442:18994 [0] NCCL INFO NET/Socket : Using [0]enp13s0f0:172.17.4.53<0> [1]br-dae752dcb0f0:172.24.0.1<0> [2]veth5b540b7:fe80::804d:6cff:fed2:a18a%veth5b540b7<0> [3]vethe5714be:fe80::74db:44ff:fed8:a269%vethe5714be<0> [4]veth39f21f7:fe80::6889:c5ff:fe9d:c70d%veth39f21f7<0> [5]vetha449a19:fe80::ce:5cff:feee:3f14%vetha449a19<0> [6]veth652d145:fe80::6c19:1bff:fe94:7970%veth652d145<0> [7]vetha93427a:fe80::4058:9dff:fe06:d483%vetha93427a<0>
soft:18442:18994 [0] NCCL INFO Using network Socket
soft:18442:18996 [2] NCCL INFO Using network Socket
soft:18442:18997 [3] NCCL INFO Using network Socket
soft:18442:18998 [4] NCCL INFO Using network Socket
soft:18442:18999 [5] NCCL INFO Using network Socket
soft:18442:19000 [6] NCCL INFO Using network Socket
soft:18442:19001 [7] NCCL INFO Using network Socket
soft:18442:18995 [1] NCCL INFO Us

2024-01-06 07:17:17.438118: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe4cc10af70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-06 07:17:17.438171: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2024-01-06 07:17:17.438182: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2024-01-06 07:17:17.438189: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (2): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2024-01-06 07:17:17.438196: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (3): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2024-01-06 07:17:17.438203: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (4): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2024-01-06 07:17:17.438211: I te

782/782 [==============================] - ETA: 0s - loss: 1.5239 - accuracy: 0.4511INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
782/782 [==============================] - 38s 23ms/step - loss: 1.5239 - accuracy: 0.4511 - val_loss: 1.2781 - val_accuracy: 0.5422
Epoch 2/10
782/782 [==============================] - 16s 20ms/step - loss: 1.1762 - accuracy: 0.5869 - val_loss: 1.0877 - val_accuracy: 0.6189
Epoch 3/10
782/782 [==============================] - 16s 20ms/step - loss: 1.0348 - accuracy: 0.6381 - val_loss: 0.9948 - val_accuracy: 0.6565
Epoch 4/10
782/782 [==============================] - 16s 20ms/step - loss: 0.9408 - accuracy: 0.6727 - val_loss: 0.9768 - val_accuracy: 0.6559
Epoch 5/10
782/782 [==============================] - 16s 20ms/step - l

KeyboardInterrupt: 

In [3]:
os.environ['PATH'] ,os.environ['CUDA_HOME']

('/data4/conda_envs/g2/bin:/usr/local/cuda-11.6/bin:/home/beihang/.vscode-server/bin/0ee08df0cf4527e40edc9aa28f4b5bd38bbff2b2/bin/remote-cli:/usr/bin:/data4/conda_envs/g2/bin:/sbin:/usr/local/cuda-10.1/bin:/home/beihang/anaconda3/bin:/home/beihang/anaconda3/condabin:/bin:/usr/bin:/usr/local/bin:/usr/local/cuda-10.1/bin:/home/beihang/anaconda3/bin:/bin:/usr/bin:/bin:/usr/bin:/usr/local/bin:/usr/local/cuda-10.1/bin:/home/beihang/anaconda3/bin:/bin:/usr/bin:/usr/local/cuda/bin:/snap/bin',
 '/usr/local/cuda-11.6')

In [4]:
import tensorflow as tf

if tf.test.is_built_with_cuda():
    print("TensorFlow can access CUDA")
else:
    print("TensorFlow cannot access CUDA")

TensorFlow can access CUDA
